In [3]:
import pandas as pd
import os
from tqdm import tqdm

# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

# initialise progress_apply
tqdm.pandas(desc="my bar!")

In [4]:
# Import date and drop useless column
path = os.getcwd()

data = pd.read_csv('{}/data_assignment_1.csv'.format(path))

data.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
# take a first look at the number of NAs
print(data.isna().sum())
initial_length = len(data)

id              0
year            0
yearest     67916
industry    79437
pertot      79437
enggrad     73439
sales       79541
va          79835
gom         79843
rdint       79720
reext       79544
ipnc        81662
ipnf        81665
ipnm        81660
ipr         79454
patent      79504
dtype: int64


In [6]:
# loop to fill the NAs for each company
for i in tqdm(data['id'].unique()):

    if data.loc[data['id'] == i, ['industry']].mode(axis=0, dropna=True).empty:
        # the rows are deleted if the column is full of NAs
        data.drop(data[data['id'] == i].index, inplace=True)

    elif data.loc[data['id'] == i, ['yearest']].mode(axis=0, dropna=True).empty: 
        # the rows are deleted if the column is full of NAs
        data.drop(data[data['id'] == i].index, inplace=True)
        
    else:
        # fill NAs with the variable mode for industry and yearest (makes more sense to our minds)
        data.loc[data['id'] == i, ['industry']] = data.loc[data['id'] == i, ['industry']].mode(axis=0, dropna=True).iloc[0, 0]
        data.loc[data['id'] == i, ['yearest']] = data.loc[data['id'] == i, ['yearest']].mode(axis=0, dropna=True).iloc[0, 0]

        # interpolate the columns where variables are continuous
        continuous_variables = ['enggrad', 'sales', 'va', 'gom','rdint', 'reext']
        data.loc[data['id'] == i, continuous_variables] = data.loc[data['id'] == i, continuous_variables].interpolate(method='linear', axis=0, limit_direction='both')

        # interpolate the columns where variables are discrete
        discrete_variables = ['pertot', 'patent']
        data.loc[data['id'] == i, discrete_variables] = data.loc[data['id'] == i, discrete_variables].interpolate(method='linear', axis=0, limit_direction='both').round(0)

        # fill NAs in binary variables
        binary_variables = ['ipnc', 'ipnf', 'ipnm', 'ipr']
        data.loc[data['id'] == i, binary_variables] = data.loc[data['id'] == i, binary_variables].interpolate(method='linear', axis=0, limit_direction='both')

100%|██████████| 5304/5304 [02:35<00:00, 34.03it/s]


In [7]:
len_after_cleaning = len(data)
print('number of observations:', len_after_cleaning, '\n')
print(initial_length - len_after_cleaning, 'rows have been deleted during the process', '\n')
print('number of NAs for each column:', '\n', data.isna().sum())

number of observations: 106030 

15962 rows have been deleted during the process 

number of NAs for each column: 
 id             0
year           0
yearest        0
industry       0
pertot         0
enggrad      322
sales        276
va           483
gom          483
rdint         92
reext        115
ipnc        5681
ipnf        5681
ipnm        5681
ipr           23
patent        23
dtype: int64


In [8]:
# Create age variable
data.insert(loc=3, column='age', value=data.apply(lambda row: row['year'] - row['yearest'], axis=1), allow_duplicates=True)

In [9]:
# Compute the sales growth rate
data['sales_growth_rate'] = data.sort_values('year').groupby('id')['sales'].pct_change(fill_method=None)

# Compute the 90th percentile of the sales growth rate for each year
percentile_90_thresholds = data.groupby('year')['sales_growth_rate'].quantile(0.9)

# Create a binary target variable based on the 90th percentile
data['is_hgf'] = data.progress_apply(lambda row: 1 if row['sales_growth_rate'] >= percentile_90_thresholds[row['year']] else 0, axis=1)

C:\Users\EMCCd\AppData\Local\Temp\ipykernel_14976\3459315293.py:2: FutureWarning: The 'fill_method' and 'limit' keywords in SeriesGroupBy.pct_change are deprecated and will be removed in a future version. Call ffill before calling pct_change instead.
  data['sales_growth_rate'] = data.sort_values('year').groupby('id')['sales'].pct_change(fill_method=None)
my bar!: 100%|██████████| 106030/106030 [00:01<00:00, 92692.60it/s]


In [10]:
# drop the remaining NAs to perform model training and predictions
data = data.dropna()
print(len_after_cleaning - len(data), 'more rows have been deleted')

10506 more rows have been deleted


In [11]:
# change data types

data_types_dict = {'id': int,
                   'year': int,
                   'yearest': int,
                   'age': int,
                   'industry': int,
                   'pertot': int,
                   'enggrad': float,
                   'sales': float,
                   'va': float,
                   'gom': float,
                   'rdint': float,
                   'reext': float,
                   'ipnc': bool,
                   'ipnf': bool,
                   'ipnm': bool,
                   'ipr': bool,
                   'patent': int,
                   'sales_growth_rate': float,
                   'is_hgf': bool}

data = data.astype(data_types_dict)

In [12]:
# checking for missing values
data.isnull().sum()

id                   0
year                 0
yearest              0
age                  0
industry             0
pertot               0
enggrad              0
sales                0
va                   0
gom                  0
rdint                0
reext                0
ipnc                 0
ipnf                 0
ipnm                 0
ipr                  0
patent               0
sales_growth_rate    0
is_hgf               0
dtype: int64

In [13]:
data

,id,year,yearest,age,industry,pertot,enggrad,sales,va,gom,rdint,reext,ipnc,ipnf,ipnm,ipr,patent,sales_growth_rate,is_hgf
1,1,1991,1971,20,12,314,1.30,"19,476,182.25","8,034,809.00",12.25,"282,475.72",0.00,False,False,False,True,0,0.43,True
2,1,1992,1971,21,12,332,1.30,"25,369,490.50","8,671,159.00",10.20,"348,587.05",0.00,False,False,False,True,0,0.30,True
3,1,1993,1971,22,12,349,1.30,"31,262,798.75","9,307,509.00",8.15,"414,698.38",0.00,False,False,False,True,0,0.23,True
4,1,1994,1971,23,12,366,2.70,"37,156,107.00","9,943,859.00",6.10,"480,809.70",0.00,False,False,False,True,0,0.19,False
5,1,1995,1971,24,12,365,2.70,"42,619,415.00","12,599,852.00",10.90,"286,442.40","350,991.10",False,False,False,True,0,0.15,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106462,4629,2008,1994,14,11,30,0.00,"4,461,571.00","1,417,235.00",10.60,0.00,0.00,False,False,False,True,0,0.00,False
106463,4629,2009,1994,15,11,34,0.00,"5,310,000.00","1,953,751.00",17.40,0.00,0.00,False,False,False,False,0,0.19,True
106464,4629,2010,1994,16,11,29,0.00,"5,648,950.00","8,601,262.00",34.80,0.00,0.00,False,False,False,False,0,0.06,False
106465,4629,2011,1994,17,11,23,0.00,"5,054,727.00","9,107,937.00",16.40,0.00,0.00,False,False,False,False,0,-0.11,False


In [14]:
#Change the True_False values of the is_hgf column to 1 and 0
data['is_hgf'] = data['is_hgf'].astype(int)

In [15]:
# checking the distribution of Target Varibale
data['is_hgf'].value_counts()

is_hgf
0    79978
1    15546
Name: count, dtype: int64

In [16]:
data.groupby('is_hgf').mean()

,id,year,yearest,age,industry,pertot,enggrad,sales,va,gom,rdint,reext,ipnc,ipnf,ipnm,ipr,patent,sales_growth_rate
is_hgf,,,,,,,,,,,,,,,,,,
0,"2,441.60","2,000.88","1,977.91",22.97,10.00,195.12,4.73,"42,083,023.36","14,696,331.93",5.67,"301,167.99","154,287.48",0.12,0.11,0.12,0.32,0.18,-0.02
1,"2,057.56","2,004.71","1,977.30",27.41,10.27,230.64,5.15,"58,650,710.18","30,219,832.17",7.25,"471,622.60","294,788.95",0.12,0.11,0.12,0.32,0.24,0.50


In [29]:
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [30]:
X = data.drop(columns='is_hgf', axis=1)
Y = data['is_hgf']

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)


In [32]:
print(X.shape, X_train.shape, X_test.shape)

(95524, 18) (71643, 18) (23881, 18)


In [58]:
# Model training 
#Logistic Regression 
model = LogisticRegression()
# training the Logistic Regression model using Training data

model.fit(X_train, Y_train)


LogisticRegression()

In [59]:
#Model Evaluation :
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)  

In [60]:
print('Accuracy on training data = ', training_data_accuracy)

Accuracy on training data =  0.8370531663944838


In [55]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [61]:
print('Accuracy on test data = ', test_data_accuracy)

Accuracy on test data =  0.8376533645994724


In [62]:
# We can change the value of C and play with the flexibility of the model
logreg001 = LogisticRegression(max_iter=5000, C=0.001).fit(X_train, Y_train)
logreg50 = LogisticRegression(max_iter=5000, C=50).fit(X_train, Y_train)

print("Accuracy C=0.001 (test): {:.3f}".format(logreg001.score(X_test, Y_test)))
print("Accuracy C=50 (test): {:.3f}".format(logreg50.score(X_test, Y_test)))

Accuracy C=0.001 (test): 0.838
Accuracy C=50 (test): 0.838


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

In [ ]:
X = data.drop(['sales_growth_rate'], axis = 1)
y = data['sales_growth_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [ ]:
# Instantiate the model
ridge = Ridge(normalize = True)

# Fit the model on the training data
ridge = ridge.fit(X_train, y_train)

# Visualize coefficients
print(pd.Series(ridge.coef_, index = X.columns))

# Check the performance of the model
print('MSE (training): %.2f' % mean_squared_error(y_train, ridge.predict(X_train)),
      'MSE (test): %.2f' % mean_squared_error(y_test, ridge.predict(X_test)), sep='\n')





In [ ]:
# Set manually some values for alpha
alphas = 10**np.linspace(5,-2,100)*0.5
alphas

coefs = []
for a in alphas:
    ridge = Ridge(alpha=a, fit_intercept=False)
    ridge.fit(X_train, y_train)
    coefs.append(ridge.coef_)

#Plot ridge coefficients as a function of the regularization
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.xlabel('alpha')
plt.ylabel('coefficient')
plt.title('Ridge coefficients profile')
plt.axis('tight')
plt.show()

# We use cross-validation to choose the tuning parameter
ridgecv = RidgeCV(alphas = alphas, cv = 10, scoring = 'neg_mean_squared_error', normalize = True)
ridgecv.fit(X_train, y_train)
ridgecv.alpha_


print('MSE (training): %.2f' % mean_squared_error(y_train, ridgecv.predict(X_train)),
      'MSE (test): %.2f' % mean_squared_error(y_test, ridgecv.predict(X_test)), sep='\n')


In [ ]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)
lassocv.alpha_

# How many coeffiecient are set to zero?
print('Number of features used:', np.sum(lassocv.coef_ != 0))

print('MSE (training): %.2f' % mean_squared_error(y_train, lassocv.predict(X_train)),
      'MSE (test): %.2f' % mean_squared_error(y_test, lassocv.predict(X_test)), sep='\n')

